In [1]:
import numpy as np


In [2]:
def NN2NA(NN):
  rows,columns=NN.shape

  columns_NA=np.count_nonzero(NN)
  if(rows != columns):
    print("Malformed NN Matrix. rows are not equal to columns")
    return none,none
  k=0
  NA=np.zeros((rows,columns_NA))
  arches = ["" for i in range(columns_NA)]

  for i in range(columns):
    for j in range(rows):
      if NN[i,j]==1:
        NA[i,k]=1
        NA[j,k]=-1
        arches[k]=[int(i),int(j)]
        k+=1
  return NA, arches


def obtain_weight(C,arc_idxs,arch):
  k=arc_idxs.index(arch)
  weight=C[k]
  return weight



In [13]:
a=[1,2,3]
a[1]=a[1]+4
a

[1, 6, 3]

In [19]:
NN = np.array([[0, 1, 1, 0, 0, 0],
               [0, 0, 0, 1, 0, 1],
               [0, 0, 0, 0, 1, 0],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 0]])

#Original_C = np.array([7, 1, 2, 3, 2, 1, 2])
#C_augmenting=Original_C.copy()
#C_decreasing=[]

Original_C = [7, 1, 2, 3, 2, 1, 2]
C_augmenting=Original_C.copy()
C_decreasing=[]

Aeq, arc_idxs = NN2NA(NN)
beq = np.array([1, 0, 0, 0, 0, -1])
bounds = tuple([(0, None) for arcs in range(0, Aeq.shape[1])])

w = np.ones(NN.shape[1])*np.inf #weights initialization
w[0]=0

precedents=[None]*NN.shape[1]  #precedents initialization

#print('## Optimizer inputs ## \n'
#      'Cost vector: %s \n\n '
#      'Arches in order:\n%s \n\n'
#      'A_eq Node-Arc matrix:\n%s \n\n'
#      'b_eq demand-supply vector: %s \n\n'
#      'Bounds of each X arc variable: %s' % (C,arc_idxs,Aeq, beq, bounds))

stop=0
augmenting_arches=arc_idxs.copy()
decreasing_arches=[]
path=[None]
nodes_to_explore=[0]

count2=0

print("nodes_to_explore     ",nodes_to_explore)
print("arches_to_explore    ",augmenting_arches)
print("w                    ",w)
print("precedents           ",precedents)



#This code is a simplified version of the EX3. It implements DFS, but it is not the most optimal solution.
while path!=[NN.shape[1]-1] and stop==0:
  path=[]
  arches_to_explore=augmenting_arches.copy()
  print("--------------------------------- Finding path N°:",count2+1,"---------------------------------")

  count1=0
  min_C=0
  min_C_arch=[None,None]
  precedents=[None]*NN.shape[1]
  while (stop==0 and len(nodes_to_explore)>0 and len(arches_to_explore)>0):  #The idea is to go through all the nodes and arches. Stop is a flag to avoid infinite loops.
    next_nodes_to_explore=[]

    for node_i in nodes_to_explore: #The algorithm goes through the actual nodes to explore
      for arch_i in arches_to_explore:  #It will find the next neighbours of the actual node
        node_start=(arch_i[:][0]) #node_start has the number of the node of origin
        if node_start==node_i:
          node_end=(arch_i[:][1]) #node_end has the number of the node of destination
          precedents[node_end]=node_start  #now the precedent for the next node is the node that is explored in this iteration
          if node_end<(NN.shape[1]-1):next_nodes_to_explore.append(node_end)  #If the next node is not the last, there will be an other iteration

    nodes_to_explore=next_nodes_to_explore.copy()

    print("########## STEP",count1,"##########")
    print("nodes_to_explore     ",nodes_to_explore)
    print("arches_to_explore    ",arches_to_explore)
    print("precedents           ",precedents)

    count1+=1
    if (count1>100): #break the bucle after 100 iterations
      stop=1 
      print("Algorithm stoped because it counted 100 iterations in the second bucle!!!")

  count2+=1
  if (count2>100): #break the bucle after 100 iterations
    stop=1 
    print("Algorithm stoped because it counted 100 iterations in the first bucle!!!")
  

  print("%%%%%%%%%%%%%%%%%%%%%% RESULT %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
  k=int(NN.shape[1]-1) #The idea is to start in the last node and find the way back to the origin
  min_C=np.inf #min_C will be the maximum capacity of the path
  min_C_arch=[]
  arch=[]
  arches_in_path=[]
  while (k!=None):
    if precedents[k]!=None:
      arch=[precedents[k],k]
      if min_C>obtain_weight(C_augmenting,augmenting_arches,arch): #Search for the capacity of the path
        min_C=obtain_weight(C_augmenting,augmenting_arches,arch)
        min_C_arch=arch
      arches_in_path.insert(0,arch)
    path.insert(0,k)
    k=precedents[k]

  print("The minimum C is:",min_C,"for arch:",min_C_arch)

  for arch in arches_in_path:
    index=augmenting_arches.index(arch)
    C_augmenting[index]=C_augmenting[index]-min_C #calculate new C for each arch of the path
    if C_augmenting[index]<0: print("Check! Negative value for C_augmenting[",index,"]")  #line just for debug purposes. It is not expected to obtain a negative value.
    if C_augmenting[index]==0:  #if the result is 0 then this arch has to be deleted and decreasing arch has to be created.
      del C_augmenting[index]
      #This algorithm is not taking care about all the decreasing arches. It only saves the decreasing arches that will define max_flow
      del augmenting_arches[index]
      decreasing_arches.append([min_C_arch[:][1],min_C_arch[:][0]]) #the inverse arch is added
      C_decreasing.append(min_C)
  arches_to_explore=augmenting_arches.copy()

  print("Augmenting Arches:   ",augmenting_arches)

  print("Arches in path:",arches_in_path)
  s_out=""
  for i in path:
    if i==0:
      s_out=s_out+"s->"
    elif i==NN.shape[1]-1:
      s_out=s_out+"t"
    else:
      s_out=s_out+str(i+1)+"->"
  if path!=[NN.shape[1]-1]:
    print("Path: ",s_out)
    print("path =",path)
  else:
    print("There is no new path in this iteration")
  nodes_to_explore=[0]

  print("\n\n\n-------------------------------------------------------------------------------------")
  print("--------------------------------- Final graph ---------------------------------------")
  print("Augmenting Arches")
  print(augmenting_arches)
  print("Capacity for Augmenting Arches")
  print(C_augmenting)
  print("Decreasing Arches")
  print(decreasing_arches)
  print("Capacity for Decreasing Arches")
  print(C_decreasing)
  print("-------------------------------------------------------------------------------------------")
  print("-------------------------------------------------------------------------------------------")


print("\n\nMax Flow = ",sum(C_decreasing))

nodes_to_explore      [0]
arches_to_explore     [[0, 1], [0, 2], [1, 3], [1, 5], [2, 4], [3, 5], [4, 5]]
w                     [ 0. inf inf inf inf inf]
precedents            [None, None, None, None, None, None]
--------------------------------- Finding path N°: 1 ---------------------------------
########## STEP 0 ##########
nodes_to_explore      [1, 2]
arches_to_explore     [[0, 1], [0, 2], [1, 3], [1, 5], [2, 4], [3, 5], [4, 5]]
precedents            [None, 0, 0, None, None, None]
########## STEP 1 ##########
nodes_to_explore      [3, 4]
arches_to_explore     [[0, 1], [0, 2], [1, 3], [1, 5], [2, 4], [3, 5], [4, 5]]
precedents            [None, 0, 0, 1, 2, 1]
########## STEP 2 ##########
nodes_to_explore      []
arches_to_explore     [[0, 1], [0, 2], [1, 3], [1, 5], [2, 4], [3, 5], [4, 5]]
precedents            [None, 0, 0, 1, 2, 4]
%%%%%%%%%%%%%%%%%%%%%% RESULT %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
The minimum C is: 1 for arch: [0, 2]
Augmenting Arches:    [[0, 1], [1, 3], [1, 5], [2, 

In [5]:
type(C_augmenting)

list